# Vertical Chat
A sample how to build a chat for small business using:

* GPT 35
* Panel
* OpenAI


This is just a simple sample to start to understand how the OpenAI API works, and how to create Prompts. It Is really far from beign a complete solution.
We are going to introduce some interesting points:

* The roles in a conversation.
* How is the conversations’ memory preserved?

Deeper explanations in the article: [Create Your First Chatbot Using GPT 3.5, OpenAI, Python and Panel.](https://medium.com/towards-artificial-intelligence/create-your-first-chatbot-using-gpt-3-5-openai-python-and-panel-7ec180b9d7f2)

In [1]:
#if you need an API Key from OpenAI
#https://platform.openai.com/account/api-keys

from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [2]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

def continue_conversation(messages, temperature=0):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=temperature,
    )
    #print(str(response.choices[0].message["content"]))
    return response.choices[0].message.content

In [3]:
def add_prompts_conversation(_):
    #Get the value introduced by the user
    prompt = client_prompt.value_input
    client_prompt.value = ''

    #Append to the context the User prompt.
    context.append({'role':'user', 'content':f"{prompt}"})

    #Get the response.
    response = continue_conversation(context)

    #Add the response to the context.
    context.append({'role':'assistant', 'content':f"{response}"})

    #Update the panels to show the conversation.
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600)))

    return pn.Column(*panels)

In [4]:
#Creating the prompt
#read and understand it.
import panel as pn  # GUI

context = [ {'role':'system', 'content':"""
Act as an OrderBot, you work collecting orders in a delivery only fast food restaurant called
My Dear Frankfurt. \
First welcome the customer, in a very friendly way, then collects the order. \
You wait to collect the entire order, beverages included \
then summarize it and check for a final \
time if everything is ok or the customer wants to add anything else. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very friendly style. \
The menu includes \
burger  12.95, 10.00, 7.00 \
frankfurt   10.95, 9.25, 6.50 \
sandwich   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
martra sausage 3.00 \
canadian bacon 3.50 \
romesco sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
vichy catalan 5.00 \
"""} ]

#Creating the panel.
pn.extension()

panels = []

client_prompt = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="talk")

interactive_conversation = pn.bind(add_prompts_conversation, button_conversation)

dashboard = pn.Column(
    client_prompt,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True),
)

# dashboard
# dashboard.servable()

# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong: if the system prompt is too generic, the model will often hallucinate and/or taking decisions on it's own (for example, if you don't define the sizes for the burguers, the model may pick one of the prices without asking the customer)

 - What did you learn?
    - **System prompts must be clear, specific, and intentionally designed**. The quality and structure of the system prompt significantly influence the behavior, and reliability of the model. Ambiguity or vagueness in instructions can lead to inconsistent or unintended responses.
    - **Creating a basic, functional prompt is relatively straightforward**. However, developing a robust chatbot that consistently performs well across diverse scenarios —such as handling varied customer interactions, producing clear and accurate responses, and leveraging available tools effectively— requires more effort. This typically involves multiple iterations, careful testing, and ongoing adjustments to align the model's behavior with the desired outcomes.

<br>

## Setup...

In [5]:
#
# Let's first define some reusable logic for the chatbot.
# (we'll use it for the following iterations, including different system prompts and using tools)
#


import json


def process_chat_interaction(_, tools=[]):

    prompt = client_prompt.value_input
    client_prompt.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=context,
        tools=tools,
        temperature=0.1,
        max_tokens=300
    )

    # If the response includes a tool call, handle it
    if response.choices[0].message.tool_calls is not None:

        tool_call = response.choices[0].message.tool_calls[0]
        tool_name = tool_call.function.name
        tool_args = json.loads(tool_call.function.arguments)

        # Verify tool is declared
        declared_tool_names = {tool["function"]["name"] for tool in tools}
        if tool_name in declared_tool_names:
            tool_result = process_customer_complaint(**tool_args, conversation=str(context))

            context.append(response.choices[0].message)
            context.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": str(tool_result)
            })

            response = client.chat.completions.create(
                model="gpt-4o",
                messages=context,
            )
            assistant_msg = response.choices[0].message.content
        else:
            # Tool not defined, continue with the assistant's response
            assistant_msg = response.choices[0].message.content

    # If no tool call, just get the assistant's response
    else:
        assistant_msg = response.choices[0].message.content

    # Append the assistant's response to the context & update the panels
    context.append({'role':'assistant', 'content':f"{assistant_msg}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(assistant_msg, width=600)))
    return pn.Column(*panels)



def generate_chatbot(system_prompt, tools=[]):
    """
    Generates an interactive chatbot interface using Panel.

    Parameters:
    - system_prompt (list): The initial list of messages to start the conversation context. Should include a system message (e.g., role='system').
    - tools (list): Optional list of OpenAI tool/function definitions to enable tool use during the conversation.
    """
    global context, panels, client_prompt, button_conversation, interactive_conversation

    context = system_prompt  # Initialize context with the provided system prompt
    panels = []  # Reset panels

    client_prompt = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
    button_conversation = pn.widgets.Button(name="talk")

    interactive_conversation = pn.bind(lambda _: process_chat_interaction(_, tools), button_conversation)

    dashboard = pn.Column(
        client_prompt,
        pn.Row(button_conversation),
        pn.panel(interactive_conversation, loading_indicator=True),
    )

    # dashboard.servable()
    return dashboard # returnining the dashboard, so that it can be displayed from other cells


<br>

## Version 1: Improve Restauran System Prompt

In [ ]:
#
# Version 1: Improve Restauran System Prompt
#

system_prompt_A = [ {'role':'system', 'content':"""
You are OrderBot, a friendly assistant working at 'My Dear Frankfurt', a delivery-only fast food restaurant.

Your tasks:
1. Warmly welcome the customer.
2. Collect their complete order — food and beverages.
3. Clarify all options, sizes, and extras to ensure items are uniquely identified from the menu.
4. Summarize the order and confirm it with the customer.
5. Ask if they want to add anything else.
6. Collect the payment (card reader is unfortunately out of service so we can only accept cash).

Always respond in a short, very friendly tone.

Menu:

Main Items (Large, Medium, Small):
- Burger: 12.95, 10.00, 7.00
- Frankfurt: 10.95, 9.25, 6.50
- Sandwich: 11.95, 9.75, 6.75

Sides:
- Fries: 4.50 (large), 3.50 (small)
- Salad: 7.25

Toppings:
- Extra cheese: 2.00
- Mushrooms: 1.50
- Martra sausage: 3.00
- Canadian bacon: 3.50
- Romesco sauce: 1.50
- Peppers: 1.00

Drinks (Large, Medium, Small):
- Coke: 3.00, 2.00, 1.00
- Sprite: 3.00, 2.00, 1.00
- Vichy Catalan: 5.00 (single size)
                     
If the customer asks for a menu item that is not listed, politely inform them that it is not available and suggest alternatives from the menu.

If the customer has any complaints or issues, address them with empathy and provide a solution if possible. If you cannot resolve the issue, inform the customer in a kind manner that you will pass their feedback to the management and ask if they want to proceed with their order.

"""} ]



In [7]:
dashboard = generate_chatbot(system_prompt_A)

dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'11ea5877-5e0f-4003-a376-8813e857f881': {'version…

<br>

## Version 2: System Prompt for an Online Sports Store

In [8]:

#
# Version 2: System Prompt for an Online Sports Store
#

system_prompt_B = [ {'role':'system', 'content':"""
You are GearBot, a helpful assistant at 'Peak Performance', an online sports equipment and apparel store.

Your tasks:
1. Greet the customer warmly.
2. Help them browse or find specific products — equipment, apparel, or accessories.
3. Clarify product options (sizes, colors, models, categories) and provide details as needed.
4. Summarize their cart and confirm their order.
5. Ask if they'd like to add anything else.
6. Inform them that checkout is online via card or bank transfer.

Always respond in a short, upbeat, and supportive tone.
Keep interactions efficient and to the point — minimize unnecessary messages.


Categories:

Apparel:
- Running Shoes (Men/Women): €89.99
- Sportswear T-Shirt: €29.99
- Shorts: €39.99
- Running Jacket: €49.99

Equipment:
- Dumbbell (8kg): €15.00
- Yoga Mat: €35.00
- Resistance Band: €12.00
- Mountain Bicycle: €499.00
- Road Bicycle: €699.00

Accessories:
- Water Bottle: €9.99
- Sports Watch: €79.00
- Gym Bag: €29.99

If the customer asks for something that isn't listed, kindly inform them it's unavailable and suggest alternatives from the catalog.

If the customer has a complaint or issue, respond with empathy. Try to resolve it or, if needed, assure them their concern will be passed to support. Ask if they'd still like help placing an order.
"""} ]


In [9]:
dashboard = generate_chatbot(system_prompt_B)

dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'abfe3380-8695-44f3-993c-3675e52e4fdd': {'version…

<br>

## Version 3: Adding Tools to our Chatbot

In [ ]:

#
# Version 3: Adding Tools to our Chatbot
#

import uuid


def process_customer_complaint(title, complaint_type, urgency, description, conversation):

    # Generate a unique ticket ID (for this example, we'll just generate a random one)
    request_id = f"CPL-{uuid.uuid4().hex[:8].upper()}"

    complaint = {
        "title": title,
        "complaint_type": complaint_type,
        "urgency": urgency,
        "description": description,
        "conversation": conversation,
        "status": "pending",
        "request_id": request_id
    }

    # For this example, we'll just print the complaint to simulate logging it,
    # in a real use case, we can save it to a database or send it to a support system.

    print(f"Complaint logged: [{urgency}] {request_id} - {title} 📌")

    return {
        "status": "received",
        "summary": f"Request '{title}' has been received successfully.",
        "request_id": request_id
    }

In [12]:
tools_list = [{
    "type": "function",
    "function": {
        "name": "process_customer_complaint",
        "description": "Log and categorize a customer's complaint, including full context, for escalation to the management team.",
        "parameters": {
            "type": "object",
            "properties": {
                "title": {
                    "type": "string",
                    "description": "A short summary of the complaint"
                },
                "complaint_type": {
                    "type": "string",
                    "enum": ["quality", "price", "service", "payment", "other"],
                    "description": "Type of the complaint"
                },
                "urgency": {
                    "type": "string",
                    "enum": ["Low", "Medium", "High Priority", "Unknown"],
                    "description": "Urgency level of the complaint"
                },
                "description": {
                    "type": "string",
                    "description": "A longer summary of the complaint"
                }
            },
            "required": ["title", "complaint_type", "urgency", "description", "conversation"]
        }
    }
}]

In [13]:

extra_instruction = """
If a customer expresses a complaint or issue respond to the customer with empathy and try to understand the situation.

If the customer insists and/or is very dissatisfied, use the tools available to report the issue to the management team.
"""


# Let's put it together and generate the chatbot with the tools and extra instructions.
system_prompt_B[0]['content'] += extra_instruction
dashboard = generate_chatbot(system_prompt_B, tools=tools_list)

dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'b1bb1cd4-d0c4-45cb-8737-bb9b92a59006': {'version…

Result:

In the tests, our model used the tools provided to log 2 customer complaints,

- Complaint logged: [Medium] CPL-36FA3D4D - Yoga Mat Quality Issue 📌
- Complaint logged: [Low] CPL-0EFF3FE1 - Bike Price Feedback 📌